<a href="https://colab.research.google.com/github/Kabongosalomon/RDC-Mobongoli/blob/main/jw300_utils/building_global_test_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Setting Up the data

Downloading the global test set is simple,
we need to set english and your target as source and target language, then we find the intersection of the english test set with the target corpus after that we get the corresponding target sentences from the target corpus.

In [2]:
%%capture
!pip install opustools-pkg

# SET THE LANGUAGE CODE and other variables.

You need to change the value below for your language!

The language codes from the [JW300 corpus website](https://object.pouta.csc.fi/OPUS-JW300/v1/languages.json) are: 
```
{
    "language": "French - Français",
    "language_en": "French",
    "language_native": "Français",
    "language_short": "fr",
    "url": "https://wol.jw.org/fr/wol/pref/r30/lp-f?newrsconf=r30&newlib=lp-f&url="
}, 

{
    "language": "Lingala - Lingala",
    "language_en": "Lingala",
    "language_native": "Lingala",
    "language_short": "ln",
    "url": "https://wol.jw.org/ln/wol/pref/r126/lp-li?newrsconf=r126&newlib=lp-li&url="
},

{
    "language": "Tshiluba - Tshiluba",
    "language_en": "Tshiluba",
    "language_native": "Tshiluba",
    "language_short": "lua",
    "url": "https://wol.jw.org/lua/wol/pref/r477/lp-sh?newrsconf=r477&newlib=lp-sh&url="
},

{
    "language": "Kikongo - Kikongo",
    "language_en": "Kikongo",
    "language_native": "Kikongo",
    "language_short": "kwy",
    "url": "https://wol.jw.org/kwy/wol/pref/r128/lp-kg?newrsconf=r128&newlib=lp-kg&url="
},

{
    "language": "Swahili (Congo) - Kiswahili (Congo)",
    "language_en": "Swahili (Congo)",
    "language_native": "Kiswahili (Congo)",
    "language_short": "swc",
    "url": "https://wol.jw.org/swc/wol/pref/r143/lp-zs?newrsconf=r143&newlib=lp-zs&url="
  },
```
Already-created test sets: https://raw.githubusercontent.com/ai-drc/RDC-Mobongoli/main/jw300_utils/test/

In [3]:
import os
source_language = "fr"
target_language = "ln" # TODO: CHANGE THIS TO YOUR LANGUAGE! "ha" is hausa. See the language codes at https://opus.nlpl.eu/JW300.php
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.|
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# No need to use gdrive since we are training on gcp
!mkdir -p "$src-$tgt-$tag"
os.environ["gdrive_path"] = "%s-%s-%s" % (source_language, target_language, tag) # saving directly on the vm

In [4]:
!echo $gdrive_path

fr-ln-baseline


#### Downloading the corpus data

for precaution , am removing the old data

In [5]:
!rm -f w300.$src jw300.$tgt JW300_latest_xml_$src-$tgt.xml.gz JW300_latest_xml_$src-$tgt.xml JW300_latest_xml_$src.zip  JW300_latest_xml_$tgt.zip test.fr-any.fr

In [ ]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/fr-ln.xml.gz not found. The following files are available for downloading:

   6 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/fr-ln.xml.gz
 278 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/fr.zip
  60 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/ln.zip

 345 MB Total size
./JW300_latest_xml_fr-ln.xml.gz ... 100% of 6 MB
./JW300_latest_xml_fr.zip ... 100% of 278 MB
./JW300_latest_xml_ln.zip ... 100% of 60 MB


In [ ]:
! wget https://raw.githubusercontent.com/ai-drc/RDC-Mobongoli/main/jw300_utils/test/test.$src-any.$src

  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

In [ ]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = f"test.{source_language}-any.{source_language}"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

In [ ]:
!ls

#### Building the corpus

In the 2 cells below you can check if the 2 datasets are aligned. Even if you don't speak the language you can get a sense, especially with similar words, punctuation, and so forth.

In [ ]:
! head -5 jw300.$src

In [ ]:
! head -5 jw300.$tgt

In [ ]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language  ## source language is english
target_file = 'jw300.' + target_language ## Target is whatever you set. For our example it was ha, so jw300.ha
target_test = {}
source = []
target = []
english_sentences_in_global_test_set = {}  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as src_f:
    for i, line in enumerate(src_f):
        # Skip sentences that are contained in the test set and add them into the new frencg test
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            # Here is the intersection with the global test set
            english_sentences_in_global_test_set[i] = line.strip()           
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in english_sentences_in_global_test_set.keys():
            target.append(line.strip())
        else:
            #Collecting the aligned test sentences
            target_test[j] = line.strip()
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(english_sentences_in_global_test_set.keys()), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(10)

## Check a random item!
Let's pick one of the keys in the dictionary at random and check it. 

In [ ]:
import random
keys_in_target_test = list(target_test.keys())
print(type(keys_in_target_test))
random_key = random.choice(keys_in_target_test)
print(f"The random key we picked was {random_key}")

In [ ]:
target_test[random_key]

In [ ]:
english_sentences_in_global_test_set[random_key]

Do the two look like they line up? 

## Check several rows at the tail end

Let's get a sample from the end of the dataset

In [ ]:
target_test_set = pd.DataFrame(zip(target_test.values(), english_sentences_in_global_test_set.values()), columns=[f'{target_language}_equivalent', f'{source_language}_equivalent'])

In [ ]:
target_test_set = target_test_set.reset_index()

In [ ]:
target_test_set = target_test_set.set_index("index")

In [ ]:
target_test_set.tail()

Removing duplicates from english and target set

In [ ]:
target_test_set = target_test_set.drop_duplicates(subset=f'{target_language}_equivalent')

In [ ]:
target_test_set = target_test_set.drop_duplicates(subset=f'{source_language}_equivalent')

In [ ]:
target_test_set.head()

In [ ]:
target_test_set.shape

In [ ]:
target_test_set.loc[~target_test_set[f'{source_language}_equivalent'].isin(en_test_sents)]

## Write out target-language test set file
In our example, we should have `test.ln-any.ln`, but it will be different for you if you picked a different code.

In [ ]:
target_test_filename = f"test.{target_language}-any.{target_language}"
print(target_test_filename)

## Write out English-language test set file
In our example, we should have `test.fr-ln.fr`, but it will be different for you if you picked a different code.

**Make sure the data lines up in the two files!**
The first line of each file should be translations of each other.


In [ ]:

with open(target_test_filename, "w") as test_tgt_any_tgt:
    test_tgt_any_tgt.write("\n".join(target_test_set[f'{target_language}_equivalent']))

In [ ]:
!head -5 test.$tgt-any.$tgt

In [ ]:
source_test_filename = f"test.{source_language}-{target_language}.{source_language}"
print(f"saving english aligned sentences to {source_test_filename}")
with open(source_test_filename, "w") as test_src_tgt_src:
    test_src_tgt_src.write("\n".join(target_test_set[f'{source_language}_equivalent']))
!ls -al

In [ ]:
!head -5 test.$src-$tgt.$src

In [ ]:
source_test_filename = f"test.{source_language}-{target_language}.{target_language}"
print(f"saving english aligned sentences to {source_test_filename}")
with open(source_test_filename, "w") as test_en_tgt_tgt:
    test_en_tgt_tgt.write("\n".join(target_test_set[f'{target_language}_equivalent']))
!ls -al

In [ ]:
!head -5 test.$src-$tgt.$tgt

## One last check to see if the two files are aligned

Let's just get one more sample! Let's take from the end this time

In [ ]:
!echo "test.$src-$tgt.$src"
!tail -5 test.$src-$tgt.$src
!echo
!echo "**********************"
!echo "test.$src-$tgt.$tgt"
!tail -5 test.$src-$tgt.$tgt
!echo
!echo "**********************"
!echo "test.$tgt-any.$tgt"
!echo "**********************"
!tail -5 test.$tgt-any.$tgt